In [1]:
from bdsg.bdsg import HashGraph
from bdsg.bdsg import SnarlDistanceIndex
from bdsg.bdsg import PackedGraph

#### STEP 1: IMPORT THE DATA 
1. The graph in PackedGraph format (using ```vg convert -p``` ) 
2. The distance index for the snarl/chain tree

In [2]:
graph_path: str = 'test/test_graph.vg'
index_path: str = 'test/test_idx.dist'

In [3]:
graph = PackedGraph()
graph.deserialize(graph_path)
# graph.deserialize('test.vg')


idx = SnarlDistanceIndex()
idx.deserialize(index_path)
# idx.deserialize('idx.dist')

##### TEST 1: CHECK HOW THE TREE STRUCTURE WORKS. 
1. In theory the root of the tree is going to be a chain.
2. I iterate over its children and if I find a snarl, I check if it is a leaf snarl.
3. Each snarl has as a child a chain. If the children of the chain are not snarls, it is a leaf snarl.
4. Else if it has other snarls, I keep iterating until I find the leaf snarl.


Which functions do I need?
1. from a chain, traverse its children. 
2. If find a snarl. Iterate over its children (chains(s)).
3. If the childern of the chain of the snarl are all nodes, append the snarl to the list;
4. Else go to 2.

In [4]:
root_handle = idx.get_root()

In [5]:
leaf_snarls = [] 
contains_child_snarls = False
num_nodes = 0

def check_for_snarl(child_net_handle):
    global contains_child_snarls
    global num_nodes
    if idx.is_snarl(child_net_handle):
        contains_child_snarls = True
    elif idx.is_node(child_net_handle):
        num_nodes += 1
    return True

# THIS FUNCTION TAKES A SNARL. FOR EACH CHILD (CHAIN) OF THE SNARL, CHECK THEIR CHILDREN. 
# IF NO ONE HAS A SNARL, THE SNARL IS A LEAF
def snarl_iteratee(handle):
    global contains_child_snarls
    contains_child_snarls = False
    snarl_children = []
    idx.for_each_child(handle, lambda y: snarl_children.append(y) or True) 
    
    num_nodes = 0
    for s_c in snarl_children:
        idx.for_each_child(s_c, check_for_snarl)
    
    if ((not contains_child_snarls) and (num_nodes < 10)):
        leaf_snarls.append(handle)
    return True
idx.traverse_decomposition(snarl_iteratee, lambda x: True, lambda y: True)
# THIS FUNCTION TAKES A SNARL. IF IT HAS MORE THAN 1 CHILD, IT MEANS IT IS A LEAF
# THIS FROM THE INTUITION THAT AN INTERNAL SNARLS HAS AS 1 CHILD THAT IS A CHAIN OF NODES AND SNARL(S)
# NOT SURE IT IS TRUE ALWAYS
def snarl_inf(handle):
    snarl_children = []
    idx.for_each_child(handle, lambda y: snarl_children.append(y) or True)
    
    if len(snarl_children) > 1:
        leaf_snarls_inf.append(handle)
    return True


leaf_snarls_inf = [] 
idx.traverse_decomposition(snarl_inf, lambda x: True, lambda y: True)

print('printing out')
for el in leaf_snarls:
    print(idx.net_handle_as_string(el))
    start_bound = idx.get_start_bound(el)
    end_bound = idx.get_end_bound(el)

    # Inspect the orientations
    print(f"Start Bound ID: {graph.get_id(idx.get_handle(start_bound, graph))}, is_reverse: {graph.get_is_reverse(idx.get_handle(start_bound, graph))}")
    print(f"End Bound ID: {graph.get_id(idx.get_handle(end_bound, graph))}, is_reverse: {graph.get_is_reverse(idx.get_handle(end_bound, graph))}")
    
for el in leaf_snarls_inf:
    print(idx.net_handle_as_string(el))

printing out
simple snarl 6rev->5revtraversing start->end
Start Bound ID: 6, is_reverse: True
End Bound ID: 5, is_reverse: True
snarl 5rev->3revtraversing start->end
Start Bound ID: 5, is_reverse: True
End Bound ID: 3, is_reverse: True
simple snarl 6rev->5revtraversing start->end
snarl 5rev->3revtraversing start->end


#### STEP 2: GENERATE THE ANCHOR DICTIONARY
1. Traverse the SNARL TREE, using ```index.traverse_decomposition```
2. When detecting a LEAF SNARL, PASS IT TO THE SNARL CONSTRUCTION
3.  When detecting a LEAF SNARL:
    1. The snarl has to contain less than X = 10 elements; X is a parameter. 
    2. The number of paths passing by the snarl has to be > MIN and < MAX. Both parameters 
    3. For each path in the snarl, 

In [6]:
import sys
sys.path.append('./assembler')
from anchor import SnarlAnchor

In [7]:
anchoring = SnarlAnchor(10)

In [8]:
anchoring.build_graph(graph_path, index_path)

In [9]:
leaf_snarl_net_handles: list = anchoring.process_snarls()

Visiting snarl 2rev->1revtraversing start->end
Children: node 6rev has #nodes: 1 and has_snarls is: False
Children: simple snarl 6rev->5revtraversing start->end has #nodes: 1 and has_snarls is: True
Children: node 5rev has #nodes: 2 and has_snarls is: True
Children: snarl 5rev->3revtraversing start->end has #nodes: 2 and has_snarls is: True
Children: node 3rev has #nodes: 3 and has_snarls is: True
Visiting simple snarl 6rev->5revtraversing start->end
Children: node 9fd has #nodes: 1 and has_snarls is: False
Children: node 10fd has #nodes: 2 and has_snarls is: False
Visiting snarl 5rev->3revtraversing start->end
Children: node 4fd has #nodes: 1 and has_snarls is: False
Children: node 7fd has #nodes: 2 and has_snarls is: False
Children: node 8fd has #nodes: 3 and has_snarls is: False


In [10]:
for el in leaf_snarl_net_handles:
    print(idx.net_handle_as_string(el[0]))
    for n in el[1]:
        print(idx.net_handle_as_string(n), end=",")
    print()

simple snarl 6rev->5revtraversing start->end
node 9fd,node 10fd,
snarl 5rev->3revtraversing start->end
node 4fd,node 7fd,node 8fd,


In [11]:
leaf_snarl_net_handles

[(<bdsg.handlegraph.net_handle_t at 0x7f4c9b70d1b0>,
   <bdsg.handlegraph.net_handle_t at 0x7f4cbc2e85b0>]),
 (<bdsg.handlegraph.net_handle_t at 0x7f4c9b70ec70>,
   <bdsg.handlegraph.net_handle_t at 0x7f4cbc2e8bb0>])]

In [12]:
anchoring.print_tree_structure()

Chain: chain 2rev->1revtraversing start->end
Node: node 2rev
Snarl: snarl 2rev->1revtraversing start->end
Chain: chain 6rev->3revtraversing start->end
Node: node 6rev
Snarl: simple snarl 6rev->5revtraversing start->end
Chain: node 9fd pretending to be a chain in a simple snarl
Node: node 9fd
Chain: node 10fd pretending to be a chain in a simple snarl
Node: node 10fd
Node: node 5rev
Snarl: snarl 5rev->3revtraversing start->end
Chain: node 4fd pretending to be a chain
Node: node 4fd
Chain: node 7fd pretending to be a chain
Node: node 7fd
Chain: node 8fd pretending to be a chain
Node: node 8fd
Node: node 3rev
Node: node 1rev


In [13]:
one_snarl = leaf_snarl_net_handles[1][0]
# Get the start and end bounds (which are net handles)
start_bound = idx.get_start_bound(one_snarl)
end_bound = idx.get_end_bound(one_snarl)
print(f"Start bound is {idx.net_handle_as_string(start_bound)}")

# Get their corresponding node handles and orientations
start_node_id = idx.node_id(start_bound)
start_node_handle = idx.get_handle(start_bound,graph)
end_node_id = idx.node_id(end_bound)
end_node_handle = idx.get_handle(end_bound,graph)

# # Optionally, check if they are in reverse orientation
start_is_reverse = graph.get_is_reverse(start_node_handle)
end_is_reverse = graph.get_is_reverse(end_node_handle)

# # Output the results
start_direction = "reverse" if start_is_reverse else "forward"
end_direction = "reverse" if end_is_reverse else "forward"

print(f"Start node of snarl: {start_node_id}, Direction: {start_direction}")
print(f"End node of snarl: {end_node_id}, Direction: {end_direction}")
# print(start_node_id, end_node_id)

Start bound is sentinel of snarl 5rev->3revtraversing start->end
Start node of snarl: 5, Direction: reverse
End node of snarl: 3, Direction: reverse


In [14]:
steps_on_start_node = []
graph.for_each_step_on_handle(start_node_handle, lambda y: steps_on_start_node.append(y) or True)

True

In [15]:
steps_on_start_node

In [16]:
path_handles = []
path_names = []
for step in steps_on_start_node:
    path_handle = graph.get_path_handle_of_step(step)
    path_handles.append(path_handle)
    path_name = graph.get_path_name(path_handle)
    path_names.append(path_name)


In [17]:
path_names

['P10#10#chr1#0',
 'P9#9#chr1#0',
 'P8#8#chr1#0',
 'P7#7#chr1#0',
 'P6#6#chr1#0',
 'P5#5#chr1#0',
 'P4#4#chr1#0',
 'P1#1#chr1#0']

In [22]:
path_1_handle = path_handles[0]
path_1_name = path_names[0]

sentinels = [graph.get_id(end_node_handle), graph.get_id(start_node_handle)] \
    if start_is_reverse else [graph.get_id(start_node_handle), graph.get_id(end_node_handle)]
print(sentinels[0],sentinels[1])
print(f"Walking on path {path_1_name}")

# Step 3: Traverse all the steps on the path using for_each_step_in_path
def traverse_step(step_handle):
    # For each step, get the corresponding node handle (handle_t)
    node_handle = graph.get_handle_of_step(step_handle)
    # Get information about this node
    node_id = graph.get_id(node_handle)
    print(f"visiting {node_id}", end=": ")
    global keep
    if not keep and node_id not in sentinels:
        print('skipped.')
        return True
    
    node_sequence = graph.get_sequence(node_handle)
    is_reversed = graph.get_is_reverse(node_handle)
    traversal.append((node_id,not(is_reversed)))
    direction = "reverse" if is_reversed else "forward"
    print('visited.')
    
    # Output the node details and whether it's traversed forward or reverse
    print(f"Node {node_id}, Sequence: {node_sequence}, Traversed in {direction} direction")
    if keep and node_id in sentinels:
        print('ending visit.')
        keep = False
        return False
    
    keep = True
    return True  # Continue traversing



# Traverse the entire path and process each step
traversals = []
traversal = []
keep = False

for path_h in path_handles:
    graph.for_each_step_in_path(path_h, traverse_step)
    traversals.append(traversal)
    traversal = []

3 5
Walking on path P10#10#chr1#0
visiting 2: skipped.
visiting 6: skipped.
visiting 10: skipped.
visiting 5: visited.
Node 5, Sequence: NN, Traversed in reverse direction
visiting 7: visited.
Node 7, Sequence: CCC, Traversed in reverse direction
visiting 4: visited.
Node 4, Sequence: NNNNN, Traversed in reverse direction
visiting 3: visited.
Node 3, Sequence: GGGG, Traversed in reverse direction
ending visit.
visiting 1: skipped.
visiting 3: visited.
Node 3, Sequence: CCCC, Traversed in forward direction
visiting 4: visited.
Node 4, Sequence: NNNNN, Traversed in forward direction
visiting 7: visited.
Node 7, Sequence: CCC, Traversed in reverse direction
visiting 5: visited.
Node 5, Sequence: NN, Traversed in forward direction
ending visit.
visiting 1: skipped.
visiting 3: visited.
Node 3, Sequence: CCCC, Traversed in forward direction
visiting 4: visited.
Node 4, Sequence: NNNNN, Traversed in forward direction
visiting 7: visited.
Node 7, Sequence: GGG, Traversed in forward direction


In [23]:
def print_traversal(trv):
    for node in trv:
        direction = ">" if node[1] == True else "<"
        print(f"{direction}{node[0]}",end="")
    print()

for pt,trv in zip(path_names,traversals):
    print(f"{pt}: ",end="")
    print_traversal(trv)

P10#10#chr1#0: <5<7<4<3
P9#9#chr1#0: >3>4<7>5
P8#8#chr1#0: >3>4>7>5
P7#7#chr1#0: >3>4>8>5
P6#6#chr1#0: >3>4>8>5
P5#5#chr1#0: >3>4>7>5
P4#4#chr1#0: >3>4>7>5
P1#1#chr1#0: >3>5


In [24]:
def get_anchor_id(traversal):
    anchor = traversal[len(traversal)//2][0]
    return anchor
        

In [25]:
for trv in traversals:
    print(get_anchor_id(trv), end=" ")
    print_traversal(trv)

4 <5<7<4<3
7 >3>4<7>5
7 >3>4>7>5
8 >3>4>8>5
8 >3>4>8>5
7 >3>4>7>5
7 >3>4>7>5
5 >3>5


In [38]:
def is_equal_path(path_1,path_2):
    #if different length, false
    if len(path_1) != len(path_2):
        return False
    
    #start at 0
    pos_path_1 = 0 if path_1[0][1] else len(path_1) - 1
    pos_path_2 = 0 if path_2[0][1] else len(path_2) - 1
    for i in range(len(path_1)):
        start1,orientation1 = path_1[pos_path_1]
        start2,orientation2 = path_2[pos_path_2]
        if start1 == start2 and orientation1 == orientation2:
            continue
        else: return False
    return True

In [44]:
paths_dict = dict()
for trv in traversals:
    anchor = get_anchor_id(trv)
    print(f"{anchor}:", end=" ")
    print_traversal(trv)
    
    if anchor not in paths_dict:
        paths_dict[anchor] = [trv]
    else:
        possible_paths = paths_dict[anchor]
        insert = True
        for pts in possible_paths:
            if is_equal_path(trv,pts):
                insert = False
                print("not accepted")
                break
        if insert:
            paths_dict[anchor].append(trv)
        
        
print(paths_dict)

4: <5<7<4<3
7: >3>4<7>5
7: >3>4>7>5
not accepted
8: >3>4>8>5
8: >3>4>8>5
not accepted
7: >3>4>7>5
not accepted
7: >3>4>7>5
not accepted
5: >3>5
{4: [[(5, False), (7, False), (4, False), (3, False)]], 7: [[(3, True), (4, True), (7, False), (5, True)]], 8: [[(3, True), (4, True), (8, True), (5, True)]], 5: [[(3, True), (5, True)]]}
